# Lab 06 - Statistics
Tyler Bevan

## Unit 47

In [63]:
import pandas as pd
alco = pd.read_csv('niaaa.csv')
alco = alco.fillna(method='ffill')
# I changed mine to have a multiindex so the years are an index.
alco["Year"] = alco["Year"].map(lambda x : int(x))
multi = pd.MultiIndex.from_tuples(list(zip(alco['State'], alco['Year'])), names=["State", "Year"])
alco.index=multi
alco = alco.drop(columns=["State", "Year"])

beer_seriesNY = alco.loc['New York']["Beer"]
beer_seriesCA = alco.loc['California']["Beer"]

beer_seriesNY.corr(beer_seriesCA)

0.9709778539165477

In [37]:
beer_seriesCA.cov(beer_seriesNY)

0.01743816287878787

In [38]:
[x.skew() for x in (beer_seriesCA, beer_seriesNY)]

[0.16457291292976317, 0.32838100586311286]

In [40]:
frameNY = alco.loc['New York']
frameNY.skew()

Beer       0.328381
Wine       0.127308
Spirits    0.656699
dtype: float64

In [41]:
frameNY.corr()

,Beer,Wine,Spirits
Beer,1.000000,0.470690,0.908969
Wine,0.470690,1.000000,0.611923
Spirits,0.908969,0.611923,1.000000


In [42]:
frameNY.cov()

,Beer,Wine,Spirits
Beer,0.016103,0.002872,0.026020
Wine,0.002872,0.002312,0.006638
Spirits,0.026020,0.006638,0.050888


In [121]:
pop = pd.read_csv('pop2009.csv', usecols=(0,1,2,3,4,5,6,7,8,9,10)).dropna(how="all")
pop["State"] = pop["State"].map(lambda x: str(x).lstrip('.'))
pop.set_index("State", inplace=True)
# Get rid of commas in numbers
for n in range(2000,2010):
    pop[str(n)] = pop[str(n)].map(lambda x: int(str(x).replace(",", "")))
# Force years to be a column and numeric
pop = pop.stack().reset_index().rename(index=str, columns={"level_1": "Year", 0: "Population"})
pop["Year"] = pop["Year"].map(lambda x : int(x))
# Make a multi index
multi = pd.MultiIndex.from_tuples(list(zip(pop['State'], pop['Year'])), names=["State", "Year"])
pop.index=multi
# Remove duplicate columns
pop = pop.drop(columns=["State", "Year"])

In [122]:
pop_seriesNY = pop.loc["New York"].loc[2009:2000]["Population"]
frameNY.loc[2009:2000].corrwith(pop_seriesNY)

Beer      -0.520878
Wine       0.936026
Spirits    0.957697
dtype: float64

In [130]:
from scipy.stats import pearsonr
pop_sorted = pop_seriesNY.sort_index()
alco_10 = alco.loc['New York'].loc[2009:2000].sort_index()
corrs = [(bev,) + pearsonr(alco_10[bev], pop_sorted)
         for bev in alco_10.columns]
pd.DataFrame(corrs, columns=("bev", "r", "p-value")).set_index("bev")

,r,p-value
bev,,
Beer,-0.520878,0.122646
Wine,0.936026,0.000068
Spirits,0.957697,0.000013


In [146]:
alco2009 = alco.reset_index().set_index("Year").loc[2009].reset_index().set_index("State").drop(columns=["Year"])
alco2009.corr()

,Beer,Wine,Spirits
Beer,1.000000,-0.031560,0.452279
Wine,-0.031560,1.000000,0.599791
Spirits,0.452279,0.599791,1.000000


In [148]:
pearsonr(alco2009["Wine"], alco2009["Beer"])

(-0.031560488300856844, 0.825984813107873)

## Twenty First Century S&P 500
Data pulled directly from yahoo finance archive.

In [153]:
sp500 = pd.read_csv("sp500.csv").set_index("Date")
sp500.describe()

,Open,High,Low,Close,Adj Close,Volume
count,4465.000000,4465.000000,4465.000000,4465.000000,4465.000000,4.465000e+03
mean,1492.019816,1500.373571,1483.048869,1492.259445,1492.259445,3.169026e+09
std,508.352807,508.558871,508.196271,508.496805,508.496805,1.466269e+09
min,679.280029,695.270020,666.789978,676.530029,676.530029,3.560700e+08
25%,1131.920044,1140.400024,1124.719971,1132.010010,1132.010010,1.834920e+09
50%,1312.829956,1319.739990,1304.719971,1312.939941,1312.939941,3.236620e+09
75%,1873.250000,1879.750000,1863.260010,1872.250000,1872.250000,3.975280e+09
max,2936.760010,2940.909912,2927.110107,2930.750000,2930.750000,1.145623e+10


In [160]:
sp500["Close"].skew()

0.9785274256532506

In [161]:
sp500["Volume"].skew()

0.7023794595895168

In [162]:
sp500["Close"].corr(sp500["Volume"])

0.14169018622670598

The correlation is close to zero, thus it is unlikely that the close value and the volume are not reliably related.